In [1]:
from os.path import join as pj
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
import nltk
nltk.download('wordnet')
from sklearn.model_selection import train_test_split
np.random.seed(400)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Data with english abstract and published in 2020

In [2]:
path = 'C:/Users/tr/CodeVsCovid19/data'
file_name = 'metadata_new_new.csv'
df = pd.read_csv(pj(path, file_name))
df_en_20 = df.copy()
df_en_20 = df[df['language']=='en']
#df_en_20 = df_en[df_en.publish_time.str[:4]=='2020']

In [3]:
df_en_20.drop('Unnamed: 0', axis=1, inplace=True)

C:\Users\tr\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [5]:
df_en_20.dropna(subset=['abstract'], inplace=True)

C:\Users\tr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
df_en_20.shape

(37135, 18)

In [23]:
df_en_20.abstract.isna().sum()

29

In [31]:
df_en_20

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url,language
0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/001727,en
1,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/003889,en
2,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/006866,en
3,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/007476,en
4,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/010389,en
5,z3tgnzth,c41fdb2efd6d61384a92a84cbba3f8233629a41b,biorxiv,The infant airway microbiome in health and dis...,10.1101/012070,NaN,NaN,biorxiv,The nasopharynx (NP) is a reservoir for microb...,2014-12-02,Shu Mei Teo; Danny Mok; Kym Pham; Merci Kusel;...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/012070,en
6,1xxrnpg3,1dd898b5ca1ae70ec0e3cad89fc87a165002a99e,biorxiv,Using heterogeneity in the population structur...,10.1101/017178,NaN,NaN,biorxiv,"In 2013, U.S. swine producers were confronted ...",2015-03-27,Eamon B. O’Dea; Harry Snelson; Shweta Bansal,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/017178,en
7,8ilzm51q,33565294e6bc67fb7ee14dcae6cfdb08148f4ea5,biorxiv,"Big city, small world: Density, contact rates,...",10.1101/018481,NaN,NaN,biorxiv,Macroscopic descriptions of populations common...,2015-04-27,Moritz U. G. Kraemer; T. Alex Perkins; Derek A...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/018481,en
8,wafvnbdu,3461d71f6890f7e5ba53bf168be3945cdb16d901,biorxiv,MERS-CoV recombination: implications about the...,10.1101/020834,NaN,NaN,biorxiv,Recombination is a process that unlinks neighb...,2015-06-12,Gytis Dudas; Andrew Rambaut,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/020834,en
9,4xocqn6o,1f9d3f9a1a0e8db6a086e0a2b5ba50cf9f235dae,biorxiv,On the causes of evolutionary transition:trans...,10.1101/027722,NaN,NaN,biorxiv,A pattern in which nucleotide transitions are ...,2015-09-28,Arlin Stoltzfus; Ryan W. Norris,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/027722,en


In [38]:
df_en_20.dropna(subset=['abstract'], inplace=True)

C:\Users\tr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
df_en_20.shape

(37100, 18)

Deal with duplicates

In [18]:
df_en_20 = df_en_20.drop_duplicates(['title','abstract'])

In [19]:
df_en_20.shape

(37129, 18)

Train on abstract

In [65]:
train, test = train_test_split(df_en_20.abstract, test_size=0.1, random_state=42)

In [41]:
def lemmatize_stemming(text):
    return PorterStemmer().stem(WordNetLemmatizer().lemmatize(text, pos='v'))
# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

processed_docs=[]
for t in df_en_20.abstract:
    processed_docs.append(preprocess(t))
    
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 8, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [42]:
lda_model.save('lda_3.model')

Results

In [43]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic {}: \nWords: {}".format(idx, topic ))
    print("\n")

Topic 0: 
Words: 0.020*"mice" + 0.013*"lung" + 0.010*"protect" + 0.008*"antigen" + 0.008*"tissu" + 0.008*"anti" + 0.008*"inflammatori" + 0.006*"cytokin" + 0.006*"mous" + 0.005*"dose"


Topic 1: 
Words: 0.015*"sensit" + 0.011*"diagnost" + 0.009*"genet" + 0.009*"prrsv" + 0.008*"rapid" + 0.008*"molecular" + 0.007*"real" + 0.007*"reaction" + 0.007*"approach" + 0.006*"tool"


Topic 2: 
Words: 0.024*"sar" + 0.011*"domain" + 0.009*"peptid" + 0.008*"fusion" + 0.008*"recombin" + 0.007*"mutat" + 0.007*"amino" + 0.007*"proteas" + 0.007*"receptor" + 0.007*"residu"


Topic 3: 
Words: 0.022*"calv" + 0.016*"speci" + 0.013*"diarrhea" + 0.012*"pedv" + 0.010*"porcin" + 0.009*"pig" + 0.008*"bovin" + 0.008*"intestin" + 0.007*"rotaviru" + 0.007*"bat"


Topic 4: 
Words: 0.019*"mer" + 0.015*"children" + 0.015*"hospit" + 0.015*"pneumonia" + 0.011*"symptom" + 0.009*"sar" + 0.009*"day" + 0.008*"ill" + 0.007*"common" + 0.007*"mortal"


Topic 5: 
Words: 0.014*"signal" + 0.014*"regul" + 0.014*"inhibit" + 0.013*"pa

Model 3 :

-Topic 0: lung, tissu inflammation, cytokin ==> symptomes

-Topic 1: diagnostic, tools, prrsv => pcr pour la grippe porcine

-Topic 2: mutation, receptor, côté protéique 

-Topic 3: grippe porcine, origine des animaux, bat 

-Topic 4: maladie, patients, les enfants, hopitaux

-Topic 5: cellule du virus

-Topic 6: cat, felin (coronavirus), contamination (particles, resistance à la température ou non, exposure, eau), présence dans le sang

-Topic 7: Epidémie, origine de chine, propagation aux autres pays, SARS...

Model 2 :
-Topic 0: chest lung images 'nucleic' 

-Topic 1: patients, evolution de la maladie

-Topic 2: nation, contact, travel, pandemie

-Topic 3: gene genome autres maladies (grippe) diagnostic

-Topic 4: genetique

-Topic 5: vaccin

-Topic 6: Hubei

-Topic 7: medicaments

Model 1
-Topic 0: Hubei, la quarantaine, contamination par le voyage (fermeture des frontières), prediction, éviter le contact social 

-Topic 1: Mortalité, Femmes enceintes dangerosité ? et enfants, asymptomatiques ?

-Topic 2: Chest and Lung Scans, lesions on images, opac => traces sur les images (grind ??)

-Topic 3: Nombres de cellules cytokin, lymphocyt dans le sang, blessure 

-Topic 4: vaccin, gene, genome etc.

-Topic 5: Tests medicaments

-Topic 6: Autres maladies, personnes à risque, diagnostic

-Topic 7: Corps médical et management de la crise 

Test

In [75]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(test[37380]))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic {}: {}".format(score, index, lda_model.print_topic(-1)))

Score: 0.9831529855728149	 Topic 0: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"


In [74]:
test[37380]

'Abstract Only a month after the outbreak of pneumonia caused by 2019-nCoV, more than forty-thousand people were infected. This put enormous pressure on the Chinese government, medical healthcare provider, and the general public, but also made the international community deeply nervous. On the 25th day after the outbreak, the Chinese government implemented strict traffic restrictions on the area where the 2019-nCoV had originated—Hubei province, whose capital city is Wuhan. Ten days later, the rate of increase of cases in Hubei showed a significant difference (p = 0.0001) compared with the total rate of increase in other provinces of China. These preliminary data suggest the effectiveness of a traffic restriction policy for this pandemic thus far. At the same time, solid financial support and improved research ability, along with network communication technology, also greatly facilitated the application of epidemic prevention measures. These measures were motivated by the need to provi

In [73]:
test

37380    Abstract Only a month after the outbreak of pn...
36175    While the coronavirus death rate may be lower ...
570      An outbreak of novel betacoronavirus, SARS-CoV...
380      Objective: To investigate the correlation betw...
1788     As of 28 February 2020, Italy had 888 cases of...
1469     In the recent issues of IRGEE, we have been as...
26151    Abstract The National Bioforensic Analysis Cen...
520      AbstractA new coronavirus SARS-CoV-2, recently...
1991     Objective To understand the possible transmiss...
402      AbstractThe 2013-2016 West Africa EBOV epidemi...
37167    A novel bat-origin coronavirus emerged in Wuha...
164      Cases from the ongoing outbreak of atypical pn...
1597     As the world is witnessing the epidemic of COV...
34317    Abstract Importation and transmission of measl...
452      SUMMARYIn the analysis of genomic sequence dat...
1924     Objective The purpose of this review of COVID-...
734      Background.This research aims to analyze the c.

In [76]:
test[570]

'An outbreak of novel betacoronavirus, SARS-CoV-2 (formerly named 2019-nCoV), began in Wuhan, China in December 2019 and the COVID-19 disease associated with infection has since spread rapidly to multiple countries. Here we report the development of SARS-CoV-2 DETECTR, a rapid (~30 min), low-cost, and accurate CRISPR-Cas12 based lateral flow assay for detection of SARS-CoV-2. We validated this method using contrived reference samples and clinical samples from infected US patients and demonstrated comparable performance to the US CDC SARS-CoV-2 real-time RT-PCR assay.'

In [77]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(test[570]))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic {}: {}".format(score, index, lda_model.print_topic(-1)))

Score: 0.5852038860321045	 Topic 6: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"
Score: 0.19930173456668854	 Topic 4: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"
Score: 0.1786642223596573	 Topic 0: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"


In [78]:
test[380]

'Objective: To investigate the correlation between clinical characteristics and cardiac injury of COVID-2019 pneumonia. Methods: In this retrospective, single-center study, 41 consecutive corona virus disease 2019 (COVID-2019) patients (including 2 deaths) of COVID-2019 in Beijing Youan Hospital, China Jan 21 to Feb 03, 2020, were involved in this study. The high risk factors of cardiac injury in different COVID-2019 patients were analyzed. Computed tomographic (CT) imaging of epicardial adipose tissue (EAT) has been used to demonstrate the cardiac inflammation of COVID-2019. Results：Of the 41 COVID-2019 patients, 2 (4.88%), 32 (78.05%), 4 (9.75%) and 3 (7.32%) patients were clinically diagnosed as light, mild, severe and critical cases, according to the 6th guidance issued by the National Health Commission of China. 10 (24.4%) patients had underlying complications, such as hypertension, CAD, type 2 diabetes mellites and tumor. The peak value of TnI in critical patients is 40-fold more

In [79]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(test[380]))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic {}: {}".format(score, index, lda_model.print_topic(-1)))

Score: 0.8044670820236206	 Topic 3: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"
Score: 0.1229139044880867	 Topic 0: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"
Score: 0.06313582509756088	 Topic 2: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"


In [86]:
train[2041]

"The new coronavirus pneumonia has been listed as one of the Class B infectious disease but is managed as Class A infectious disease. To prevent and control its spread in hospitals, the outpatient department is the first key gate. Based on the relevant diagnosis and treatment strategies of the National Health Commission of the People's Republic of China, combined with the actual situation of the hospital's epidemic prevention and control work, this article formulated comprehensive prevention and control strategies from the perspective of the patients and staffs. From the aspects of organization and leadership, medical epidemic prevention, pre-screening and screening, process formulation, admission management, cleaning and disinfection, epidemic report, prevention and control supervision, personnel and material deployment, patient education, comprehensive management, personnel management and psychological support and so on, advice and guidance on prevention and control of this infectiou

In [87]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(train[2041]))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic {}: {}".format(score, index, lda_model.print_topic(-1)))

Score: 0.9796350598335266	 Topic 7: 0.016*"manag" + 0.011*"staff" + 0.009*"recommend" + 0.008*"worker" + 0.007*"anxieti" + 0.007*"nurs" + 0.007*"practic" + 0.007*"psycholog" + 0.007*"experi" + 0.007*"work"


Mise en forme des résultats dans le dataframe

In [ ]:
nb_topics = 8
for i in range(nb_topics):
    df_en_20['topic_'+repr(i)]=0.0
i=0
idx = df_en_20.index
for a in df_en_20.abstract:
    bow_vector = dictionary.doc2bow(preprocess(a))
    for topic, score in lda_model[bow_vector]:
        df_en_20['topic_'+repr(topic)].loc[idx[i]] = score
    i+=1

C:\Users\tr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\tr\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\tr\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#i

In [130]:
df_en_20.to_csv('metadata_en_new_topics.csv', index=False)

In [133]:
df_en_20

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,url,language,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7
84,2vecg9op,09c9fcabc66a106e01ef42247cbd86b6d85bd67f,biorxiv,Mono-ADP-ribosylation by ARTD10 restricts Chik...,10.1101/2020.01.07.896977,NaN,NaN,biorxiv,AbstractA subset of intracellular mono-ADP-rib...,2020-01-08,...,https://doi.org/10.1101/2020.01.07.896977,en,None,None,None,None,None,0.135202,0.0219877,0.83746
85,2f9nc2to,b80d9de7750d627a0189fcd2dfd82c7efcb1c079,biorxiv,An interaction network of RNA-binding proteins...,10.1101/2020.01.08.899146,NaN,NaN,biorxiv,"AbstractDuring Drosophila oogenesis, the local...",2020-01-09,...,https://doi.org/10.1101/2020.01.08.899146,en,None,None,None,None,0.256521,None,None,0.729568
86,8iet67u3,e18620d5cff022d18aa23da2fa4634331fa54860,biorxiv,Altered polarization of PAR-2 signaling during...,10.1101/2020.01.09.900555,NaN,NaN,biorxiv,ABSTRACTBackgroundProtease-activated receptor ...,2020-01-09,...,https://doi.org/10.1101/2020.01.09.900555,en,0.0402769,None,None,None,None,None,None,0.950563
87,wtarrhf0,0624a12abfe85c8b5070850d912a2db4cd453236,biorxiv,TALC: Transcription Aware Long Read Correction,10.1101/2020.01.10.901728,NaN,NaN,biorxiv,ABSTRACTMotivationLong-read sequencing technol...,2020-01-11,...,https://doi.org/10.1101/2020.01.10.901728,en,None,None,None,None,0.983779,None,None,None
88,ahrrphm8,0015023cc06b5362d332b3baf348d11567ca2fbb,biorxiv,The RNA pseudoknots in foot-and-mouth disease ...,10.1101/2020.01.10.901801,NaN,NaN,biorxiv,AbstractThe positive stranded RNA genomes of p...,2020-01-11,...,https://doi.org/10.1101/2020.01.10.901801,en,None,None,None,0.191408,0.474381,0.190743,None,0.136882
89,8kced06y,f8e8a9c6f5662c1739be031e6134284ef08ea8b2,biorxiv,Programmable low-cost DNA-based platform for v...,10.1101/2020.01.12.902452,NaN,NaN,biorxiv,AbstractViral detection is critical for contro...,2020-01-16,...,https://doi.org/10.1101/2020.01.12.902452,en,None,None,None,0.802716,None,None,0.183106,None
90,sjicr24s,de3ae7971891058db2f4b36cd216531e6f533332,biorxiv,Crystal structure of the giant panda MHC class...,10.1101/2020.01.15.908608,NaN,NaN,biorxiv,ABSTRACTThe viral cytotoxic T lymphocyte (CTL)...,2020-01-16,...,https://doi.org/10.1101/2020.01.15.908608,en,None,None,None,None,None,0.992382,None,None
91,bf4qpsy7,26cb6703ca72bf97887abbc29a40b1bd9d7890f4,biorxiv,Hotspots of aberrant enhancer activity in fibr...,10.1101/2020.01.18.911297,NaN,NaN,biorxiv,AbstractFibrolamellar carcinoma (FLC) is a rar...,2020-01-18,...,https://doi.org/10.1101/2020.01.18.911297,en,None,None,None,None,None,None,None,0.981356
92,v4mbry22,8b88d01563538e4abd29c3ba493c534af30664cc,biorxiv,A mathematical model for simulating the transm...,10.1101/2020.01.19.911669,NaN,NaN,biorxiv,AbstractAs reported by the World Health Organi...,2020-01-19,...,https://doi.org/10.1101/2020.01.19.911669,en,None,None,0.359358,0.241293,None,0.380386,None,None
93,3qds5j1p,NaN,biorxiv,Genomic and protein structure modelling analys...,10.1101/2020.01.20.913368,NaN,NaN,biorxiv,AbstractDetailed genomic and structure-based a...,2020-01-21,...,https://doi.org/10.1101/2020.01.20.913368,en,None,None,None,None,0.291959,0.680213,None,None
